### Operation Scrapertron

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Attempt 1- Using Selenium to extract all UOS from CUSP
CUSP is a very useful website for displaying all unit of studies, so with Selenium, we can get the JS stuff of CUSP to load, then ask it to load every single UOS.
From there it's a matter of grabbing each individual URL

In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
import time as t

url = "https://cusp.sydney.edu.au/students/view-units-page/"
driver = webdriver.Chrome(service=ChromeService( 
	ChromeDriverManager().install())) 
driver.get(url) 
select_bar = driver.find_element("name", "unit-list_length")
select = Select(select_bar)
select.select_by_visible_text('All')
# Wait for all units to load
t.sleep(2)
unit_table = driver.find_element("id", "unit-list")
unit_rows = unit_table.find_elements('xpath', './/td')
links = []
for i in range(0, len(unit_rows), 2):
    links.append("https://www.sydney.edu.au/units/" + unit_rows[i].text)

print(links)


['https://www.sydney.edu.au/units/AERO1400', 'https://www.sydney.edu.au/units/AERO2703', 'https://www.sydney.edu.au/units/AERO2705', 'https://www.sydney.edu.au/units/AERO3260', 'https://www.sydney.edu.au/units/AERO3465', 'https://www.sydney.edu.au/units/AERO3560', 'https://www.sydney.edu.au/units/AERO3660', 'https://www.sydney.edu.au/units/AERO3760', 'https://www.sydney.edu.au/units/AERO4206', 'https://www.sydney.edu.au/units/AERO4491', 'https://www.sydney.edu.au/units/AERO4560', 'https://www.sydney.edu.au/units/AERO4591', 'https://www.sydney.edu.au/units/AERO4712', 'https://www.sydney.edu.au/units/AERO5010', 'https://www.sydney.edu.au/units/AERO5206', 'https://www.sydney.edu.au/units/AERO5210', 'https://www.sydney.edu.au/units/AERO5211', 'https://www.sydney.edu.au/units/AERO5301', 'https://www.sydney.edu.au/units/AERO5310', 'https://www.sydney.edu.au/units/AERO5400', 'https://www.sydney.edu.au/units/AERO5410', 'https://www.sydney.edu.au/units/AERO5500', 'https://www.sydney.edu.au/unit

This works, next we work on getting units with outlines.

In [4]:
# Now that we have links, we scrape those units and look for courses with outlines, this takes 6 minutes bruh
outline_links = []
for i in range(0, len(links)):
    unit_page = requests.get(links[i]).text
    unit_web_content = BeautifulSoup(unit_page)
    unit_outline_div = unit_web_content.find('div', id='currentOutlines')
    if unit_outline_div is not None:
        unit_outline_link = unit_web_content.find('div', id='currentOutlines').find_all('a')
        if unit_outline_link != []:
            print(links[i])
            outline_links += unit_outline_link


https://www.sydney.edu.au/units/AMME2450
https://www.sydney.edu.au/units/AMME9450
https://www.sydney.edu.au/units/ARCF9001
https://www.sydney.edu.au/units/ARCH9002
https://www.sydney.edu.au/units/ARCH9031
https://www.sydney.edu.au/units/ARCH9058
https://www.sydney.edu.au/units/ARCH9074
https://www.sydney.edu.au/units/ARCH9075
https://www.sydney.edu.au/units/AWSS1001
https://www.sydney.edu.au/units/AWSS2015
https://www.sydney.edu.au/units/AWSS2023
https://www.sydney.edu.au/units/BDES1012
https://www.sydney.edu.au/units/BDES3011
https://www.sydney.edu.au/units/BDES3023
https://www.sydney.edu.au/units/BIOS1155
https://www.sydney.edu.au/units/BIOS1163
https://www.sydney.edu.au/units/BIOS1167
https://www.sydney.edu.au/units/BIOS1168
https://www.sydney.edu.au/units/BIOS1170
https://www.sydney.edu.au/units/BIOS2062
https://www.sydney.edu.au/units/BIOS5090
https://www.sydney.edu.au/units/BIOS5092
https://www.sydney.edu.au/units/BMET2903
https://www.sydney.edu.au/units/BMET2925
https://www.sydn

In [7]:
# I will try with no filters for now.
final_outline_links = []
# Get the final list of links we'll look at
for link in outline_links:
    final_outline_links.append('http://sydney.edu.au'+ link['href'])

### Get data
For each assessment, get Assignment due dates, weights, and other details, as well as schoosl.

In [33]:
# This is just DATA2001 code lol
def findAssessments(URL):
    page = requests.get(URL).text
    content = BeautifulSoup(page, 'html5lib')
    assessments = content.find('div', id='assessmentDetails')
    school = content.find_all('td', class_ = "wordBreak")[1]
    try:
        headers = []
        for header in assessments.thead.find_all('th'):
            headers.append(header.text)

        data = []
        for row in assessments.find_all('tr', class_='primary'):
            assessment = {'Unit': URL.split('/')[-2], 'Session': URL.split('/')[-1]}
            for i, cell in enumerate(row.find_all(['td', 'th'])):
                if cell.find('span'):
                    assessment[headers[i]] = cell.span.text[11:22].strip()

                elif not cell.find('b'):
                    assessment[headers[i]] = cell.find(text=True, recursive=False).strip()

                else:
                    assessment[headers[i]] = cell.b.text.strip()
            
            assessment["School"] = school.text
            data.append(assessment)
    except:
        data = []

    return pd.DataFrame(data)

df = findAssessments(final_outline_links[0])

for i in range(1, len(final_outline_links)):
    uos_to_add = findAssessments(final_outline_links[i])
    df = pd.concat([df, uos_to_add])

df.to_csv("uos_duedate.csv")


C:\Users\chuah\AppData\Local\Temp\ipykernel_25256\2632002720.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  assessment[headers[i]] = cell.find(text=True, recursive=False).strip()


### Getting UOS from the Tables
New approach time, scrape all areas of study in Table S, getting links to the unit of study tables. Then from the UOS tables, get all unit codes.
Below code scrapes gets the links to unit of study tables.

In [110]:
# Everything above basically got me nowhere! it sucks!
# So, I now try to look at table S instead:
url_list = ["https://www.sydney.edu.au/handbooks/interdisciplinary_studies/table_s/subjects_ac/accounting.html", 
            "https://www.sydney.edu.au/handbooks/interdisciplinary_studies/table_s/subjects_dh/data_science.html"
            , "https://www.sydney.edu.au/handbooks/interdisciplinary_studies/table_s/subjects_io/immunology.html",
            "https://www.sydney.edu.au/handbooks/interdisciplinary_studies/table_s/subjects_pz/pathology.html"]

uos_tab_list = []

for url in url_list:
    # Get the UOS table link of the first UOS for each lettered category
    large_pages = requests.get(url).text
    content = BeautifulSoup(large_pages, 'html5')
    areas = content.find_all('li', 'hasChildren menuItem')
    active = content.find('li', 'hasChildren active menuItem')
    uos_table_link = active.find('li', "menuItem").find('a')['href']
    uos_tab_list.append(uos_table_link)
    # Get the rest
    for area in areas:
        area_page = requests.get("https://www.sydney.edu.au" + area.find('a')['href']).text
        area_content = BeautifulSoup(area_page, 'html5')
        current_page_unit = area_content.find('li', 'hasChildren active menuItem')
        uos_table_link = current_page_unit.find('li', "menuItem").find('a')['href']
        if uos_table_link not in uos_tab_list:
            uos_tab_list.append(uos_table_link)

print(uos_tab_list)
print(len(uos_tab_list))

['/content/handbooks/interdisciplinary_studies/table_s/subjects_ac/accounting/accounting_table.html', '/content/handbooks/interdisciplinary_studies/table_s/subjects_ac/american_studies/american_studies_table.html', '/content/handbooks/interdisciplinary_studies/table_s/subjects_ac/anatomy_histology/unit_of_study_table.html', '/content/handbooks/interdisciplinary_studies/table_s/subjects_ac/ancient_greek/ancient_greek_table.html', '/content/handbooks/interdisciplinary_studies/table_s/subjects_ac/ancient_history/ancient_history_table.html', '/content/handbooks/interdisciplinary_studies/table_s/subjects_ac/animal_health_disease_welfare/unit_of_study_table.html', '/content/handbooks/interdisciplinary_studies/table_s/subjects_ac/animal_production/unit_of_study_table.html', '/content/handbooks/interdisciplinary_studies/table_s/subjects_ac/anthropology/anthropology_table.html', '/content/handbooks/interdisciplinary_studies/table_s/subjects_ac/applied_medical_science/unit_of_study_table.html', 

### Getting UOS Codes
Now that we can access every single UoS table, get every single UoS code from these tables.
Once this is done we can repeat our previous approach- Check whether they have outlines first, then look at all units with outlines.

In [111]:
list_of_rows = []
for link in uos_tab_list:
    unit_page = requests.get("https://www.sydney.edu.au" + link).text
    unit_table_content = BeautifulSoup(unit_page, 'html5')
    # They didn't label the table for units and like requirements separately so I need to grab the whole thing and get only the units
    # USYD HTML Designers, why must you torment me like this
    # And something something format isn't consistent again!!
    table_checker = unit_table_content.find_all('div',"richTextModule b-single-column__container")
    # The anti-"table" measure
    if len(table_checker) > 1:
        tables = table_checker[1].find_all('table')

    else:
        tables = table_checker[0].find_all('table')

    try:
        if len(tables) > 1:
            if len(tables[1].find_all('tbody')) == 1:
                table_rows = tables[1].find_all('tbody')[0].find_all('tr')

            else:
                table_rows = tables[1].find_all('tbody')[1].find_all('tr')
        
        for row in table_rows:
            row_unit_link = row.find('strong') 
            if row_unit_link is not None:
                # print(row_unit_link)
                list_of_rows.append(row_unit_link)
    except:
        print("WTF")
        print(tables[1].find_all('tbody'))
        break


unit_urls = []
try:
    for row in list_of_rows:
        if row.find('a'):
            if row.find('a')['href'] not in unit_urls:
                print(row.find('a')['href'])
                unit_urls.append(row.find('a')['href'])
except:
    print("WTF")
    print(row)
    print(row.find('a'))


https://www.sydney.edu.au/units/BUSS1030
https://www.sydney.edu.au/units/ACCT1006
https://www.sydney.edu.au/units/ACCT2011
https://www.sydney.edu.au/units/ACCT2019
https://www.sydney.edu.au/units/ACCT3011
https://www.sydney.edu.au/units/ACCT3600
https://www.sydney.edu.au/units/ACCT3013
https://www.sydney.edu.au/units/ACCT3015
https://www.sydney.edu.au/units/ACCT3016
https://www.sydney.edu.au/units/ACCT3020
https://www.sydney.edu.au/units/ACCT3031
https://www.sydney.edu.au/units/CLAW3201
https://www.sydney.edu.au/units/INFS3110
https://www.sydney.edu.au/units/ACCT3400
https://www.sydney.edu.au/units/AMST1001
https://www.sydney.edu.au/units/AMST1202
https://www.sydney.edu.au/units/USSC1201
https://www.sydney.edu.au/units/USSC1604
https://www.sydney.edu.au/units/AMST2701
https://www.sydney.edu.au/units/AMST2606
https://www.sydney.edu.au/units/USSC2601
https://www.sydney.edu.au/units/AMST2607
https://www.sydney.edu.au/units/USSC2602
https://www.sydney.edu.au/units/ENGL2627
https://www.sydn

### Look for Outlines
Now that I managed to get all the unit links (again), repeat the same approach as above.

In [112]:
# Now that we have links, we scrape those units and look for courses with outlines, this takes 6 minutes bruh
print(len(unit_urls))
print(unit_urls)
outline_links = []
for i in range(0, len(unit_urls)):
    unit_page = requests.get(unit_urls[i]).text
    unit_web_content = BeautifulSoup(unit_page)
    unit_outline_div = unit_web_content.find('div', id='currentOutlines')
    if unit_outline_div is not None:
        unit_outline_link = unit_web_content.find('div', id='currentOutlines').find_all('a')
        if unit_outline_link != []:
            print(unit_urls[i])
            outline_links += unit_outline_link

1408
['https://www.sydney.edu.au/units/BUSS1030', 'https://www.sydney.edu.au/units/ACCT1006', 'https://www.sydney.edu.au/units/ACCT2011', 'https://www.sydney.edu.au/units/ACCT2019', 'https://www.sydney.edu.au/units/ACCT3011', 'https://www.sydney.edu.au/units/ACCT3600', 'https://www.sydney.edu.au/units/ACCT3013', 'https://www.sydney.edu.au/units/ACCT3015', 'https://www.sydney.edu.au/units/ACCT3016', 'https://www.sydney.edu.au/units/ACCT3020', 'https://www.sydney.edu.au/units/ACCT3031', 'https://www.sydney.edu.au/units/CLAW3201', 'https://www.sydney.edu.au/units/INFS3110', 'https://www.sydney.edu.au/units/ACCT3400', 'https://www.sydney.edu.au/units/AMST1001', 'https://www.sydney.edu.au/units/AMST1202', 'https://www.sydney.edu.au/units/USSC1201', 'https://www.sydney.edu.au/units/USSC1604', 'https://www.sydney.edu.au/units/AMST2701', 'https://www.sydney.edu.au/units/AMST2606', 'https://www.sydney.edu.au/units/USSC2601', 'https://www.sydney.edu.au/units/AMST2607', 'https://www.sydney.edu.au

In [125]:
# This is just DATA2001 code lol
def findAssessments(URL):
    page = requests.get(URL).text
    content = BeautifulSoup(page, 'html5lib')
    assessments = content.find('div', id='assessmentDetails')
    try:
        school = content.find_all('td', class_ = "wordBreak")[1]
        headers = []
        for header in assessments.thead.find_all('th'):
            headers.append(header.text)

        data = []
        for row in assessments.find_all('tr', class_='primary'):
            assessment = {'Unit': URL.split('/')[-2], 'Session': URL.split('/')[-1]}
            for i, cell in enumerate(row.find_all(['td', 'th'])):
                if cell.find('span'):
                    assessment[headers[i]] = cell.span.text[11:22].strip()

                elif not cell.find('b'):
                    assessment[headers[i]] = cell.find(text=True, recursive=False).strip()

                else:
                    assessment[headers[i]] = cell.b.text.strip()
            
            assessment["School"] = school.text
            data.append(assessment)
    except:
        print(content)
        print(URL)
        return None
        data = []

    return pd.DataFrame(data)

# I will try with no filters for now.
final_outline_links = []
# Get the final list of links we'll look at
for link in outline_links:
    final_outline_links.append('http://sydney.edu.au'+ link['href'])

df = findAssessments(final_outline_links[0])

for i in range(1, len(final_outline_links)):
    uos_to_add = findAssessments(final_outline_links[i])
    if uos_to_add is not None:
        print(uos_to_add['Unit'][0])
        df = pd.concat([df, uos_to_add])

df.to_csv("uos_duedate.csv")

C:\Users\chuah\AppData\Local\Temp\ipykernel_14276\1534202986.py:20: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  assessment[headers[i]] = cell.find(text=True, recursive=False).strip()


ACCT1006
ACCT2011
ACCT2019
ACCT3011
ACCT3600
ACCT3013
ACCT3016
CLAW3201
INFS3110
ACCT3400
ACCT3400
AMST1202
USSC1201
AMST2701
GCST2605
USSC3603
USSC3703
FASS3999
FASS3333
FASS3333
BIOL1008
BIOL1908
BIOL1998
CHEM1011
CHEM1111
CHEM1111
CHEM1911
MEDS1001
ANAT2008
ANAT3007
ANAT3907
ANAT3008
ANAT3908
SCPU3001
SCPU3001
GRKA1600
GRKA2600
GRKA3007
GRKA3605
ANHS1600
LATN1600
ANHS2619
ANHS2635
ARCO2007
HSTY2677
LATN2600
ANHS3636
ANTH1001
ANTH2700
ANTH2627
ANTH3604
ANTH3621
BCMB2001
BCMB2901
MEDS2003
AMED3001
AMED3901
AMED3002
ARCO1001
ARCO2109
ARCO3014
ARCO3016
ARHT1001
ARHT2616
ARHT2671
ARHT3633
ARHT3663
ARHT3681
ASNS1602
ASNS2004
ASNS2011
ASNS2618
ASNS2627
ASNS2634
CHNS2004
CHNS2613
HSTY2640
KRNS2641
ASNS3001
ASNS3616
ASNS3670
HSTY3801
BUSS1020
BUSS1020
BUSS1040
BANK2011
ECOS2004
FINC2011
FINC2011
BANK3014
BANK3400
BANK3400
BCMB3001
BCMB3901
BCMB3002
BCMB3902
PCOL3012
PCOL3912
BIOL1006
BIOL1906
BIOL1996
BIOL2030
BIOL2930
BIOL2021
BIOL2921
BIOL2029
BIOL2929
IMMU2011
IMMU2911
MICR2031
MICR2931
B